In [1]:
!pip install hyperopt


[notice] A new release of pip is available: 23.0.1 -> 23.1
[notice] To update, run: pip install --upgrade pip


In [2]:
import numpy as np
import pandas as pd
from sklearn.datasets import make_regression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from catboost import CatBoostRegressor
from hyperopt import hp, fmin, tpe, Trials
import pickle

In [3]:
space = {
    'iterations': hp.quniform('iterations', 5000, 150000, 5000),
    'learning_rate': hp.uniform('learning_rate', 0.0001, 0.1),
    'depth': hp.quniform('depth', 1, 10, 1),
    'l2_leaf_reg': hp.loguniform('l2_leaf_reg', np.log(1e-9), np.log(10)),
    'bagging_temperature': hp.loguniform('bagging_temperature', np.log(0.1), np.log(20.0)),
    'random_strength': hp.uniform('random_strength', 0, 10),
    'min_data_in_leaf': hp.quniform('min_data_in_leaf', 1, 300, 1),
    "use_best_model": True,
    "task_type": "GPU"
}

In [4]:
import os
import time
import random
import numpy as np
import torch
import torch.nn as nn
import logging
import json
seed = 42
random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True

In [5]:
test_size = 0.2
def context_data_split(data):
    """
    Parameters
    ----------
    Args:
        test_size : float
            Train/Valid split 비율을 입력합니다.
        seed : int
            랜덤 seed 값
    ----------
    """

    X_train, X_valid, y_train, y_valid = train_test_split(
                                                        data['train'].drop(['rating'], axis=1),
                                                        data['train']['rating'],
                                                        test_size=test_size,
                                                        random_state=seed,
                                                        shuffle=True
                                                        )
    data['X_train'], data['X_valid'], data['y_train'], data['y_valid'] = X_train, X_valid, y_train, y_valid
    
    # train 데이터와 validation 데이터의 인덱스 분할
    train_idx, valid_idx = train_test_split(data['train'].index, test_size=test_size, random_state=seed, shuffle=True)

    # train 데이터와 validation 데이터의 인덱스를 각각 CSV 파일로 저장
    os.makedirs('./data_index', exist_ok=True)
    data['train'].loc[train_idx].to_csv(f'./data_index/context_data_train_index.csv', index=False)
    data['train'].loc[valid_idx].to_csv(f'./data_index/context_data_valid_index.csv', index=False)
    return data

In [ ]:
with open('/opt/ml/data/20230419_015908_catboost_data.pt',"rb") as f:
    data = pickle.load(f)
del data['train']['mean_user']
del data['train']['median_user']
del data['train']['var_user']
del data['train']['std_user']
del data['train']['category_mean']
del data['train']['category_high_mean']
del data['train']['category_median']
del data['train']['category_high_median']
del data['train']['category_std']
del data['train']['category_high_std']
del data['train']['mean_user_category']
del data['train']['median_user_category']
del data['train']['var_user_category']
del data['train']['std_user_category']
del data['train']['mean_user_category_high']
del data['train']['median_user_category_high']
del data['train']['var_user_category_high']
del data['train']['user_rating_avg']
del data['train']['user_rating_var']
del data['train']['user_rating_median']
del data['train']['std_user_category_high']

del data['test']['median_user']
del data['test']['var_user']
del data['test']['std_user']
del data['test']['category_mean']
del data['test']['category_high_mean']
del data['test']['category_median']
del data['test']['category_high_median']
del data['test']['category_std']
del data['test']['category_high_std']
del data['test']['mean_user_category']
del data['test']['median_user_category']
del data['test']['var_user_category']
del data['test']['std_user_category']
del data['test']['mean_user_category_high']
del data['test']['median_user_category_high']
del data['test']['var_user_category_high']
del data['test']['user_rating_avg']
del data['test']['user_rating_var']
del data['test']['user_rating_median']
del data['test']['std_user_category_high']

data = context_data_split(data)

def rmse(real: list, predict: list) -> float:
    pred = np.array(predict)
    return np.sqrt(np.mean((real-pred) ** 2))

cat_features = ['user_id', 'isbn', 'category', 'category_high', 'publisher', 'language', 'book_author','age_map', 'location_city', 'location_state', 'location_country', 'year_of_publication_map']

def objective(params):
    model = CatBoostRegressor(
        learning_rate=params['learning_rate'],
        depth=int(params['depth']),
        l2_leaf_reg=params['l2_leaf_reg'],
        bagging_temperature=params['bagging_temperature'],
        random_strength=params['random_strength'],
        verbose=False,
        task_type="GPU",
        eval_metric='RMSE'
    )

    model.fit(data['X_train'], data['y_train'], eval_set=(data['X_valid'],data['y_valid']), cat_features=cat_features, early_stopping_rounds=300)
    y_pred = model.predict(data['X_valid'])

    rmse = np.sqrt(mean_squared_error(data['y_valid'], y_pred))
    return {'loss': rmse, 'status': 'ok'}

trials = Trials()
best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=1000,
            trials=trials)

print('Best hyperparameters:', best)

  0%|          | 0/1000 [00:00<?, ?trial/s, best loss=?]

  0%|          | 1/1000 [00:55<15:18:55, 55.19s/trial, best loss: 2.2325302571811414]

  0%|          | 2/1000 [01:05<11:35:33, 41.82s/trial, best loss: 2.1752828766156838]

  0%|          | 3/1000 [01:16<9:01:27, 32.59s/trial, best loss: 2.1752828766156838] 

  0%|          | 4/1000 [02:28<12:16:19, 44.36s/trial, best loss: 2.1378368049737975]

...

 14%|█▍        | 145/1000 [2:41:28<19:32:57, 82.31s/trial, best loss: 2.136947281689055]

 15%|█▍        | 146/1000 [2:42:48<19:23:29, 81.74s/trial, best loss: 2.136947281689055]

 15%|█▍        | 147/1000 [2:44:00<18:41:16, 78.87s/trial, best loss: 2.136947281689055]

 15%|█▍        | 148/1000 [2:45:12<18:10:26, 76.79s/trial, best loss: 2.136947281689055]

 46%|████▋     | 463/1000 [5:41:40<5:04:09, 33.98s/trial, best loss: 2.1366520454653113]

In [13]:
print('Best hyperparameters:', best)

Best hyperparameters: {'bagging_temperature': 0.10001217419400599, 'depth': 8.0, 'iterations': 75000.0, 'l2_leaf_reg': 0.0007822886554960759, 'learning_rate': 0.07161638867514175, 'min_data_in_leaf': 95.0, 'random_strength': 9.169090950443135}
